In [1]:
# =====================================================
# FAKE NEWS DETECTION – DistilBERT + PROMPT TUNING + LIAR 2
# Dataset: chengxuphd/liar2
# =====================================================

# 1. INSTALL
!pip install -q transformers datasets peft accelerate bitsandbytes scikit-learn pandas numpy psutil

# 2. IMPORT
import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from peft import PromptTuningConfig, PromptTuningInit, get_peft_model, TaskType
from google.colab import drive

warnings.filterwarnings("ignore")

# 3. GPU INFO
device_name = "CPU"
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"Device: {device_name} | VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("Device: CPU")

# 4. MOUNT GOOGLE DRIVE
try:
    drive.mount('/content/drive', force_remount=True)
except ValueError:
    print("Drive có thể đã được mount, tiếp tục...")

# 5. OUTPUT DIR
OUTPUT_DIR = "/content/drive/MyDrive/LIAR2_DistilBERT_PromptTuning"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 6. LOAD DATA (LIAR 2)
print("Loading 'chengxuphd/liar2' dataset...")
try:
    dataset = load_dataset("chengxuphd/liar2")
except Exception as e:
    print(f"Lỗi tải dataset: {e}")
    # Fallback nếu cần thiết (thường sẽ không lỗi)
    dataset = load_dataset("liar")

# Chuyển sang DataFrame
train_df = pd.DataFrame(dataset["train"])
val_df   = pd.DataFrame(dataset["validation"])
test_df  = pd.DataFrame(dataset["test"])

print("Columns detected:", train_df.columns.tolist())

# --- XỬ LÝ NHÃN (6 Lớp -> 2 Lớp) ---
# Kiểm tra xem label đã là binary chưa. Nếu max label > 1 thì cần map.
if train_df["label"].max() > 1:
    print("Mapping 6 labels -> Binary (Fake/Real)...")
    # 0: false, 1: half-true, 2: mostly-true, 3: true, 4: barely-true, 5: pants-fire
    def map_labels(label):
        if label in [0, 4, 5]: # False, Barely-True, Pants-Fire
            return 1 # FAKE
        else:                  # True, Mostly-True, Half-True
            return 0 # REAL

    train_df["label"] = train_df["label"].apply(map_labels)
    val_df["label"]   = val_df["label"].apply(map_labels)
    test_df["label"]  = test_df["label"].apply(map_labels)
else:
    print("Labels seem to be binary already.")

# --- TẠO NỘI DUNG ĐẦU VÀO ---
# Kiểm tra tên cột chứa văn bản (thường là 'statement' hoặc 'text')
text_col = "statement" if "statement" in train_df.columns else "text"
context_col = "subject" if "subject" in train_df.columns else "context"

print(f"Using text column: '{text_col}'")

def create_content(row):
    stmt = str(row.get(text_col, ""))
    spkr = str(row.get("speaker", ""))
    ctxt = str(row.get(context_col, ""))
    # Prompt Tuning cần ngữ cảnh rõ ràng
    return f"Statement: {stmt} | Speaker: {spkr} | Context: {ctxt}"

train_df["content"] = train_df.apply(create_content, axis=1)
val_df["content"]   = val_df.apply(create_content, axis=1)
test_df["content"]  = test_df.apply(create_content, axis=1)

# Tạo lại Dataset
train_dataset = Dataset.from_pandas(train_df[["content", "label"]])
val_dataset   = Dataset.from_pandas(val_df[["content", "label"]])
test_dataset  = Dataset.from_pandas(test_df[["content", "label"]])

# Đổi tên cột label cho chuẩn Transformers
if "label" not in train_dataset.column_names:
    train_dataset = train_dataset.rename_column("binary_label", "label")
    val_dataset   = val_dataset.rename_column("binary_label", "label")
    test_dataset  = test_dataset.rename_column("binary_label", "label")

dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset, "test": test_dataset})

# 7. TOKENIZER
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

tokenized = dataset_dict.map(tokenize_fn, batched=True, remove_columns=["content"])
tokenized = tokenized.rename_column("label", "labels")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 8. LOAD MODEL & CONFIG PROMPT TUNING
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# --- CẤU HÌNH FIX CỨNG CHO DISTILBERT ---
# DistilBERT: Layers=6, Hidden=768, Heads=12
n_layers = 6
hidden_size = 768
n_heads = 12

peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text="Classify validity of the statement:",
    num_virtual_tokens=64,
    tokenizer_name_or_path=MODEL_NAME,
    # Tham số bắt buộc để tránh lỗi ValueError
    num_layers=n_layers,
    token_dim=hidden_size,
    num_attention_heads=n_heads
)

model = get_peft_model(base_model, peft_config)
print("\nTham số huấn luyện (Prompt Tuning):")
model.print_trainable_parameters()

# 9. CLASS WEIGHTS
labels_array = train_df["label"].values
class_weights = compute_class_weight("balanced", classes=np.array([0,1]), y=labels_array)
class_weights = torch.tensor(class_weights, dtype=torch.float32)
print("Class Weights:", class_weights)

# 10. METRICS
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    acc = accuracy_score(labels, preds)
    probs = torch.softmax(torch.tensor(logits), dim=1)[:,1].numpy()
    try:
        auc = roc_auc_score(labels, probs)
    except:
        auc = 0.0
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1, "auc": auc}

# 11. TRAINER
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(outputs.logits, labels)
        return (loss, outputs) if return_outputs else loss

# 12. TRAINING ARGS
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=1e-2,               # LR 0.01 cho Prompt Tuning
    warmup_ratio=0.1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# 13. TRAIN
print("\n==============================")
print("🚀 TRAINING LIAR 2 - DistilBERT + PROMPT TUNING")
print("==============================\n")
trainer.train()

# 14. EVALUATE
print("\n🎯 TEST RESULTS")
results = trainer.evaluate(tokenized["test"])
for k,v in results.items():
    print(f"{k}: {v}")

# 15. SAVE
final_path = os.path.join(OUTPUT_DIR, "final_liar2_pt")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)
print(f"Saved to: {final_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.9 MB/s eta 0:00:00
Device: Tesla T4 | VRAM: 15.8 GB
Mounted at /content/drive
Loading 'chengxuphd/liar2' dataset...


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/19.0M [00:00<?, ?B/s]

valid.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Tham số huấn luyện (Prompt Tuning):
trainable params: 641,282 || all params: 67,596,292 || trainable%: 0.9487
Class Weights: tensor([0.8250, 1.2693])

🚀 TRAINING LIAR 2 - DistilBERT + PROMPT TUNING



Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Auc
1,0.695300,0.693150,0.393557,0.154887,0.393557,0.222290,0.500000
2,0.694700,0.693142,0.606443,0.367773,0.606443,0.457873,0.500000
3,0.693600,0.693163,0.393557,0.154887,0.393557,0.222290,0.500000
4,0.693300,0.693798,0.393557,0.154887,0.393557,0.222290,0.500000
5,0.693300,0.693122,0.606443,0.367773,0.606443,0.457873,0.500000



🎯 TEST RESULTS


eval_loss: 0.6931145191192627
eval_accuracy: 0.605836236933798
eval_precision: 0.367037545982105
eval_recall: 0.605836236933798
eval_f1: 0.45712948498774775
eval_auc: 0.5
eval_runtime: 2.2657
eval_samples_per_second: 1013.355
eval_steps_per_second: 31.778
epoch: 5.0
Saved to: /content/drive/MyDrive/LIAR2_DistilBERT_PromptTuning/final_liar2_pt
